# NIRCam Preimaging: Pipeline Stage 3


**Use case:** Running JWST Pipeline on NIRCam Preimaging Simulations.<br>
**Data:** JWST simulated NIRCam data from MIRAGE; LMC.<br>
**Tools:**  jwst.<br>
**Cross-intrument:** NIRCam. <br>
**Documentation:** This notebook is part of a STScI's larger [post-pipeline Data Analysis Tools Ecosystem](https://jwst-docs.stsci.edu/jwst-post-pipeline-data-analysis).<br>

## Introduction

In this Notebook we show how to create the association needed to execute calwebb_image3 and how to run calwebb_image3.

## Setting things up

In [ ]:
import os
from glob import glob
from astropy.io import fits
from astropy import wcs
from astropy.io import ascii
from astropy.table import Table


import numpy as np
import json
import yaml
from sys import exit
from shapely.geometry import Polygon 
import sys
import shapely.ops as so

from jwst.pipeline import Image3Pipeline

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as leopolygon
from matplotlib.collections import PatchCollection

In [ ]:
path='./.' # This is the working directory.

os.chdir(path)
print(os.getcwd())

In [ ]:
# Change parameters here
filtname = 'f150w'

In [ ]:
# Gather the F150W *cal.fits files back in the working directory
cwd = os.getcwd()
filter_pattern = os.path.join(cwd, '*cal.fits') 
files = glob(filter_pattern)[:] 
namelist = []
outlist = []

In [ ]:
# We convert corner pixel coordinates to world coordinates 
coord = []

for file in files:
    hdulist = fits.open(file)
    
    # Parse the WCS keywords in the primary HDU
    w = wcs.WCS(hdulist[1].header)
    
    # Print out the "name" of the WCS, as defined in the FITS header
    # print(w.wcs.name)

    # Three pixel coordinates of interest.
    # Note we've silently assumed a NAXIS=2 image here
    pixcrd = np.array([[0, 0], [2048, 0], [2048, 2048], [0, 2048]], np.float_)
    
    # Convert pixel coordinates to world coordinates
    world = w.wcs_pix2world(pixcrd, 0)
    coord.append(world)

In [ ]:
# we read the input source catalogue
pointsource_catalog = Table.read('preimaging/pointsource_LMCcalField_F150W.cat', format='ascii')
pointsource_ra = pointsource_catalog['x_or_RA']
pointsource_dec = pointsource_catalog['y_or_Dec']


In [ ]:
# We plot the field of view of the simulated images (blue) on top of the source catalogue (red)

fig = plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)

ax.scatter(pointsource_ra, pointsource_dec, c='red', label='Catalog point sources', s=0.1)

for m in range(len(files)):
    rect1 = matplotlib.patches.Polygon(coord[m], edgecolor='red', 
                                       facecolor='blue', alpha=0.2)
    ax.add_patch(rect1)
rect1.set_label('Detector')

# These limits correspond to the LMC 
plt.xlim([80.2, 80.8]) # alpha
plt.ylim([-69.4, -69.6]) # delta
plt.xlabel('RA (degrees)')
plt.ylabel('Dec (degrees)')
plt.title('F150W Mosaic')
#plt.axis('scaled')

plt.legend()
plt.show()



In order to create an association for stage 3 of the pipeline, we need to check that each image contains sources from the catalog and that the images overlap in such a way that they make a continuous mosaic.


In [ ]:
current_detector_idx = None
current_detector = None
current_detector_coords = None

number_detectors = len(files)
detectors = files[:]
detector_coords = coord[:]
referencia = None

finallist = []
finalcoords = []

for iteration in range(number_detectors):
    
    # if this is the first iteration, initialize variables
    if iteration == 0:
        current_detector_idx = 0 # select first image
        covfile = [100]
    else:
        covfile = []
        
        # calculate the overlap with all the rest of the files
        maxcoverage = 0.0
        best_overlap_index = None
        for m in range(len(detectors)):
            detector = Polygon(detector_coords[m])
            coverage = detector.intersection(referencia).area

            covfile.append(round(100 * coverage / detector.area))

            if covfile[m] >= maxcoverage: 
                maxcoverage = covfile[m]
                best_overlap_index = m
                
        if maxcoverage == 0.0:
            print('Some images are excluded')
            print('Number of excluded images: ', len(detectors))
            break
            
        # select the file with the maximum overlap
        
        # print overlap values
        # print(covfile)
        # print(best_overlap_index, covfile[best_overlap_index])
        
        # we set current index to the best overlap
        current_detector_idx = best_overlap_index

    # remove the image from list and its coordinates
    current_detector = detectors.pop(current_detector_idx)
    current_detector_coords = detector_coords.pop(current_detector_idx)
    
    referencia = Polygon(current_detector_coords) if iteration == 0 else referencia.union(Polygon(current_detector_coords))
         
    print('removing current detector ' + current_detector)
    finallist.append(current_detector)
    finalcoords.append(current_detector_coords)


    # Plot 
    # ----
    fig = plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
    ax = fig.add_subplot(111)
    
    rect1 = leopolygon([
        (80.3125, -69.4950), 
        (80.4958, -69.4392), 
        (80.6625, -69.5022), 
        (80.4792, -69.5564)], color='yellow')
    
    ax.add_patch(rect1)
    
    rect2 = leopolygon(current_detector_coords, edgecolor='navy', 
                       facecolor='navy', alpha=0.8)
    
    new_shape = so.cascaded_union(referencia)
    xs, ys = new_shape.exterior.xy
    
    ax.fill(xs, ys, alpha=0.8, fc='red', ec='red')

    ax.add_patch(rect2)
    
    plt.xlim([80.2, 80.8]) # RA
    plt.ylim([-69.4, -69.6]) # Dec
    
    plt.title('coverage = {}% \n {}'.format(str(covfile[current_detector_idx]), 
                                            current_detector), fontsize=15)
    plt.xlabel('RA (degrees)')
    plt.ylabel('Dec (degrees)')
    
    pdfname = 'figure_' + str(len(finallist)-1)
    # plt.savefig(pdfname) 
  
    plt.show()
     



Here we need to go through the final list and calculate the overlap 
with the stellar catalogue. We select the files that have an overlap of 99% or larger.



In [ ]:
bestlist = []
# these coordinates represent the extent of the LMC catalogue
referencia = Polygon([(80.3125, -69.4950), 
                      (80.4958, -69.4392), 
                      (80.6625, -69.5022), 
                      (80.4792, -69.5564)])

rejected = 0
for current_detector, current_detector_coords in zip(finallist, finalcoords):
    detector = Polygon(current_detector_coords)
    
    coverage = detector.intersection(referencia).area
    coverage = round(100 * coverage / detector.area)
    
    if coverage >= 99:
        bestlist.append(current_detector)
    else:
        rejected += 1
        
print('number of overlapping images = ', len(bestlist))
print('number of rejected images = ', rejected)

### Save Detector List to an Association (JSON file)

*Developer Note:*

There is a new way to implement the association introduced in version 0.16.0 of the JWST package. It might be worthwhile to explore.

The following link provides a complete description of the association keywords.

https://jwst-pipeline.readthedocs.io/en/latest/jwst/associations/level3_asn_technical.html#association-meta-keywords

In [ ]:
# here we print a json file with the science files in the correct order

association = {}
 
association["asn_id"] = "a3001"
association["asn_pool"] = "none" 
association["asn_rule"] = "Asn_Image" 
association["program"] = "1069" 
association["asn_type"] = "image3" 
association["constraints"] = "No constraints" 
association["code_version"] = "0.9.4"  
association["target"] = "none" 
association["version_id"] = "null" 
association["degraded_status"] = "No known degraded exposures in association."
    
products_dict = {}

products_dict["name"] = "lmc-" + filtname 
products_dict["members"] = []   
for current_detector in bestlist:
    obs_dict = {
        "expname": current_detector,
        "exptype": "science"
    }
    products_dict["members"].append(obs_dict) 

association["products"] = [products_dict] 

In [ ]:
# save association file
jsonfile = "association-" + filtname + ".json"
with open(jsonfile, 'w') as json_file:
    json.dump(association, json_file, indent=4)


# Execute the third stage of the NIRCam pipeline

Here we execute the third stage of the NIRCam pipeline. In this stage the pipeline creates a mosaic of the input images and extracts a source catalogue.

In [ ]:
# Execute pipeline
# ----------------

# run Image3 pipeline to get source catalog
im3 = Image3Pipeline()

# Options 
# -------

#im3.source_catalog.kernel_fwhm = # kernel_fwhm
#im3.source_catalog.kernel_xsize = # kernel_xsize
#im3.source_catalog.kernel_ysize = # kernel_ysize
#im3.source_catalog.npixels = # npixels
#im3.tweakreg_catalog.snr_threshold = 2000.0
#im3.resample.blendheaders = False
im3.tweakreg.skip = False
im3.tweakreg.enforce_user_order = True
im3.tweakreg.fitgeometry = 'shift' # valid values are 'shift', 'rscale', 'general'
im3.tweakreg.expand_refcat = True
im3.tweakreg.minobj = 5
im3.tweakreg.use2dhist = True
#im3.tweakreg.snr_threshold = 2000.0
im3.tweakreg.save_catalogs = True
#im3.tweakreg.xoffset = 7.5
#im3.tweakreg.yoffset = -7.2
im3.tweakreg.searchrad = 0.1 
im3.skymatch.skymethod = 'global' # valid values are: 'local', 'global', 'match', or 'global+match'
im3.source_catalog.skip = False 
im3.output_file = 'lmc-f150w-02.fits'

# Execute 
# -------

im3.run(jsonfile)